# New test with SGD without large output

In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

In [29]:
from proj1_helpers import *
import os
DATA_TRAIN_PATH = r'C:\Users\Audrey\Documents\Cours\MLrandom\data\train.csv' # TODO: download train data and supply path here 
y, tX, ids, headers = load_csv_data(DATA_TRAIN_PATH)

In [30]:
# remove this value 7343
tX = np.delete(tX, 7343, axis = 0)
y = np.delete(y, 7343, axis = 0)
ids = np.delete(ids, 7343)

Missing values replacement and normalization

In [46]:
def norm_ (feature) :
    values=[]
    for i in range (len(feature)):
        if feature[i]!=-999:
            values.append(feature[i])
    mean = np.mean(values)
    sd = np.std(values)
    return (mean, sd)

def normalize (tX) :
    tX_norm = tX
    for i in range (len(tX[0])):
        if i!=22 :
            mean, sd = norm_(tX[i])
            for j in range(len(tX)) :
                if tX[j,i]==-999 :
                    tX_norm[j,i]=0
                else :
                    tX_norm[j,i]=(tX[j,i]-mean)/sd
    return tX_norm

def cat_features (tX, num_column) :
    # For this dataset, the only categorical feature is column 22
    # The possible entries of feature PRI_jet_num are 0, 1, 2, 3
    inds0 = np.where(tX[:, num_column] == 0)
    inds1 = np.where(tX[:, num_column] == 1)
    inds2 = np.where(tX[:, num_column] == 2)
    inds3 = np.where(tX[:, num_column] == 3)

    # initialize new columns as zeros
    pri_jet0 = np.zeros(tX.shape[0])
    pri_jet1 = np.zeros(tX.shape[0])
    pri_jet2 = np.zeros(tX.shape[0])
    pri_jet3 = np.zeros(tX.shape[0])

    # set ones to appropriate columns
    pri_jet0[inds0] = 1
    pri_jet1[inds1] = 1
    pri_jet2[inds2] = 1
    pri_jet3[inds3] = 1

    # concatenate new features to tX
    tX= np.delete(tX, num_column, axis = 1)
    tX = np.column_stack((tX, pri_jet0, pri_jet1, pri_jet2, pri_jet3))
    return tX

def PCA_bias (tX) :
    m, n = tX.shape
    dataset=tX
    dataset -= tX.mean(axis=0)
    R = np.cov(dataset, rowvar=False)
    evals, evects = np.linalg.eigh(R)
    # Sorting the eigenvalues in a decreasing order
    index = np.argsort(evals)[::-1]
    evals = evals[index]
    evects = evects[:,index]
    #evects = evects[:, :n_features]
    data = np.dot(evects.T, dataset.T).T
    
    # Adding the bias term of the model
    data_ones = np.c_[np.ones((len(tX), 1)), data]
    
    return (data_ones)

def arrange_data (tX) :
    tX_prep = cat_features(normalize(tX), 22)
    data = PCA_bias(tX_prep)
    return data

In [16]:
data = arrange_data (tX) 

In [17]:
print (data.shape)

(249999, 34)


## Creation of a training and a testing set

In [23]:
def split_train_test (y, tX, ratio_train) : 
    x_test = []
    y_test = []
    x_train = []
    y_train = []
    num_row = y.shape[0]
    indices = np.random.permutation(num_row)
    limit = int(num_row*ratio_train)
    for i in range(num_row):
        if i<limit :
            x_train.append(tX[indices[i]])
            y_train.append(y[indices[i]])
        else :
            x_test.append(tX[indices[i]])
            y_test.append(y[indices[i]])
    return np.array(x_train), np.array(y_train), np.array(x_test), np.array(y_test)

x_train, y_train, x_test, y_test = split_train_test(y, data, 0.8)
print(x_train.shape)

(199999, 34)


Implementation of the functions

In [33]:
def compute_gradient(y, tx, w):
    """Compute the gradient."""
    e= y - tx.dot(w)
    X = np.transpose(tx)
    gradient = -(X.dot(e))/len(y)
    return (gradient)

def gradient_descent(y, tx, initial_w, max_iters, gamma):
    """Gradient descent algorithm."""
    # Define parameters to store w and loss
    ws = [initial_w]
    w = initial_w
    loss = compute_loss(y,tx,w)
    losses = [loss]

    for n_iter in range(max_iters):
        DL = compute_gradient(y,tx,w)
        loss = compute_loss(y,tx,w)
        w = w - gamma*DL
        ws.append(w)
        losses.append(loss)
        print("Gradient Descent({bi}/{ti}): loss={l}".format(
              bi=n_iter, ti=max_iters - 1, l=loss))

    return losses[-1], ws[-1]

In [ ]:
initial_w = np.ones(len(x_train[0]))
loss, coeffs = gradient_descent(y_train, x_train, initial_w, 100000, 0.01)

In [36]:
print (coeffs)

[-0.31408513  0.01877354 -0.01924595  0.00205096  0.08755195 -0.01404975
  0.04271845 -0.13216246 -0.2428544   0.08774975  0.12945402 -0.14163845
 -0.09373998 -0.03486062  0.04812387 -0.26259455 -0.35576724 -0.27358864
 -0.06133017 -0.04695701 -0.02005821  0.08437736  0.19218647  0.31175787
 -0.20910215  0.08748881  0.67926548 -0.46822699  0.89283116 -0.04302604
  0.25054063  1.15383636  1.00001109  1.        ]


In [39]:
def calculate_accuracy (y_test, x_test, coeffs) :
    y_pred = x_test.dot(coeffs)
    N = len (y_test)
    T = 0
    F = 0
    for i in range (N) :
        if y_test[i]*y_pred[i]>0:
            T+=1
        else : 
            F+=1
    accuracy = 100*T/N
    print("There are {acc} % of correct predictions".format(
              acc = accuracy))
    
calculate_accuracy(y_test, x_test, coeffs)

There are 73.572 % of correct predictions


In [44]:
DATA_TEST_PATH = r'C:\Users\Audrey\Documents\Cours\MLrandom\data\test.csv' # TODO: download train data and supply path here 
f, tX_test, ids_test, headers_test = load_csv_data(DATA_TEST_PATH)

In [49]:
OUTPUT_PATH = r'C:\Users\Audrey\Documents\Cours\MLrandom\data\submit.csv' # TODO: fill in desired name of output file for submission
data_test = arrange_data(tX_test)
y_pred = predict_labels(coeffs, data_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)